In [18]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [19]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [20]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [/Users/nick/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [57]:
url = 'http://www.dpdconnect.com/dailybulletin.aspx'
browser.visit(url)

In [58]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

#mainTable = soup.find('div', id='gbox_tblDB')
#listTable= soup.find('div', id="tblDB")
dateField = browser.find_by_id("mainContent_txtDate2")
dateSubmit = browser.find_by_id("mainContent_lbUpdate")
limit_selector = browser.find_by_css("#pager_center select")
print(limit_selector.value)
limit_selector.select(10000)
print(limit_selector.value)



10
10000


In [77]:
def rebaseHtml():
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    mainTable = soup.find('div', id='gbox_tblDB')
    listTable= soup.find('div', id="tblDB")
    dateField = browser.find_by_id("mainContent_txtDate2")
    dateSubmit = browser.find_by_id("mainContent_lbUpdate")
    limit_selector = browser.find_by_css("#pager_center select")
    #limit_selector.select(5)
    limit_selector.select(10000)
    limit_selector.click()
    print(limit_selector.value)


In [78]:
def changeDate(date):
    dateField = browser.find_by_id("mainContent_txtDate2")
    dateSubmit = browser.find_by_id("mainContent_lbUpdate")
    dateNow=dateField.value
    print(f'{dateNow}=>{date}')
    dateField.fill(date)
    dateSubmit.click()
    rebaseHtml()

In [79]:
rebaseHtml()
changeDate('5/28/2019')
rebaseHtml()

10000
5/28/2019=>5/28/2019
10000
10000


In [73]:
changeDate('6/6/2015')

12/25/2015=>6/6/2015


TypeError: select() got an unexpected keyword argument 'slowly'

In [66]:
changeDate('12/25/2015')

6/6/2015=>12/25/2015
10000
